In [1]:
#Create Sample & Count Tags and Keys
import xml.etree.cElementTree as ET
import pprint
import regex as re
from collections import defaultdict




# Create Sample File
OSM_FILE = "map_PHX_Metro.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm" 


# Iterative Parsing


# Count Tags
def count_tags(filename):
    tags = {}
    for _, elem in ET.iterparse(filename):
        tag = elem.tag
        if tag not in tags.keys():
            tags[tag] = 1
        else:
            tags[tag] += 1
    return tags


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(element.attrib['k']):
                keys['lower'] = keys['lower'] + 1
            elif lower_colon.search(element.attrib['k']):
                keys['lower_colon'] = keys['lower_colon'] + 1
            elif problemchars.search(element.attrib['k']):
                keys['problemchars'] = keys['problemchars'] + 1
            else:
                keys['other'] = keys['other'] + 1
    
    return keys

def key_count(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys


# Test count tags function
def test():
    tags = count_tags(OSM_FILE)
    keys = key_count(OSM_FILE)
    pprint.pprint(tags)
    pprint.pprint(keys)	

    
test()

OSM_FILE=SAMPLE_FILE

{'bounds': 1,
 'member': 13747,
 'meta': 1,
 'nd': 281901,
 'node': 228786,
 'note': 1,
 'osm': 1,
 'relation': 157,
 'tag': 153689,
 'way': 37136}
{'lower': 92690, 'lower_colon': 57758, 'other': 3241, 'problemchars': 0}


In [2]:
# Audit 
import xml.etree.cElementTree as ET
import pprint
import regex as re
from collections import defaultdict


# Audit file

# Audit street names------------------------------------------------------------------------------
# Regular expression to check for characters at end of string, including optional period.
# Eg "Street" or "St."

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)

# Common street names
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Road", "Parkway", "Commons", "Close", "Highway", "Circle", "Trail", "US"]


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)



def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

# Iterate over the osmfile and create a dictionary mapping from expected street names
# to collected streets.
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    osm_file.close()
    return street_types   


# Check Postalcodes for addresses 

# Regular expression to check whether postalcode is in appropriate format
postcode_re = re.compile('^[A-Z]{1,2}[0-9]{1,2}[A-Z]? [0-9][A-Z]{2}$') 

def is_postcode(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:postcode")


# Search for postcodes within "way" and "node"
def find_postcode():
    osm_file = open(OSM_FILE, "r")
    postcode_types = set()
    odd_postcode = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postcode(tag):
                    m = postcode_re.search(tag.attrib['v'])
                    if m:
                        postcode_types.add(tag.attrib['v'])  
                    else:
                        odd_postcode.add(tag.attrib['v'])
                        

    osm_file.close()


    return (postcode_types, odd_postcode)



# Audits
audit(OSM_FILE)



defaultdict(set,
            {'Date': {'North Date'},
             'Freeway': {'West Agua Fria Freeway', 'West Maricopa Freeway'},
             'Ki': {'West Gu u Ki'},
             'Ranchos': {'East Camino de Los Ranchos'},
             'Sol': {'West Camino del Sol'},
             'South': {'East Paradise Village Parkway South'},
             'Stewart': {'North Stewart'},
             'Sunnyvale': {'South Sunnyvale'},
             'Terrace': {'East Sandra Terrace'},
             'Verde': {'South Verde'},
             'Way': {'East Janice Way',
              'East Joseph Way',
              'North 102nd Way',
              'North 14th Way',
              'North 17th Way',
              'North 19th Way',
              'North 21st Way',
              'North 34th Way',
              'North 36th Way',
              'North 37th Way',
              'North 39th Way',
              'North 43rd Way',
              'North 55th Way',
              'North 60th Way',
              'North 7th Way',
 

In [3]:
find_postcode()

(set(),
 {'85003',
  '85004',
  '85006',
  '85007',
  '85007-4145',
  '85012',
  '85013',
  '85013-4408',
  '85015',
  '85015-3809',
  '85017',
  '85018',
  '85019',
  '85020',
  '85021',
  '85022',
  '85023',
  '85023-1508',
  '85023-2301',
  '85023-2510',
  '85023-8204',
  '85024',
  '85027',
  '85028',
  '85029',
  '85031',
  '85032',
  '85034',
  '85043',
  '85044',
  '85045',
  '85048',
  '85050',
  '85051',
  '85053',
  '85054',
  '850822',
  '85119',
  '85120',
  '85142',
  '85147',
  '85201',
  '85203',
  '85205',
  '85206',
  '85207',
  '85209',
  '85210',
  '85212',
  '85213',
  '85224',
  '85248',
  '85251',
  '85253',
  '85254',
  '85255',
  '85256',
  '85260',
  '85263',
  '85268',
  '85281',
  '85282',
  '85283',
  '85286',
  '85295',
  '85296',
  '85297',
  '85298',
  '85301',
  '85302',
  '85303',
  '85304',
  '85305',
  '85308',
  '85322',
  '85323',
  '85339',
  '85345',
  '85351',
  '85354',
  '85361',
  '85374',
  '85375',
  '85378',
  '85381',
  '85382',
  '85392',

In [4]:
# Update Street names and Postal Codes

import re
import audit


# Mapping for names to be updated
mapping = { " St ": "Street",
           "St": "Street",
            "St. ": "Street",
            "Blvd": "Boulevard",
            "Ct": "Court",
            "Dr, ": "Drive ",
            " Dr ": "Drive ",
            "Dr. ": "Drive ",
            "Rd": "Road",
            "Rd ": "Road",
            "Rd.": "Road",
            "Pl": "Place",
            "Ave": "Avenue ",
            "Ave.": "Avenue",
            "ln " : "Lane",            
            "S ": "South ",
            "S. ": " South ",
            "N ": " North ",
            "N. ": " North ",
            "W " : " West ",
            "W. ": " West ",
            "E ": "East ",
            "E. ": "East ",
            "Hwy ":"Highway ", 
           
            }

# Improving Street names
def update_name(name, mapping):
    for key in mapping.iterkeys():
        if re.search(key, name):
            name = re.sub(key, mapping[key], name)

    return name

def improve_street_name():
    st_types = audit.audit(OSM_FILE)   

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)            
            print name, "=>", better_name         
       
            #Second Check replace bad street names with corrected ones       
            if True:                 
               
                better_name = better_name.replace(" Streetewart", " Stewart")                
                print name, "=>", better_name 




# Noted a postalcode with a +4 number, will drop this and maintain only 9-digit postal code

area_postcode_re = re.compile('^[A-Z]{1,2}[0-9]{1,2}[A-Z]? ?[0-9]?$')

def update_postcode(odd_postcode):
    if area_postcode_re.search(odd_postcode):
        postcode = " "
    else:
        postcode = odd_postcode.split("-")[0]        
    return postcode
    


def improve_postcode():
    postcode_all = audit.find_postcode()

    for postcode in postcode_all[1]:
        better_postcode = update_postcode(postcode)
        #print "1: ", postcode, "=>", better_postcode
        
        
        #Second Check replace bad zip codes with correct ones 
        if True:          

            better_postcode = better_postcode.replace("850822", "85082") 
            print "2: ", postcode, "=>", better_postcode
        



# Fix Street_names
improve_street_name()
# Fix postal codes
improve_postcode()




North 55th Way => North 55th Way
North 55th Way => North 55th Way
North 14th Way => North 14th Way
North 14th Way => North 14th Way
East Joseph Way => East Joseph Way
East Joseph Way => East Joseph Way
South Larkspur Way => South Larkspur Way
South Larkspur Way => South Larkspur Way
North 17th Way => North 17th Way
North 17th Way => North 17th Way
North 43rd Way => North 43rd Way
North 43rd Way => North 43rd Way
North 37th Way => North 37th Way
North 37th Way => North 37th Way
North 21st Way => North 21st Way
North 21st Way => North 21st Way
East Janice Way => East Janice Way
East Janice Way => East Janice Way
North 39th Way => North 39th Way
North 39th Way => North 39th Way
North 34th Way => North 34th Way
North 34th Way => North 34th Way
North 19th Way => North 19th Way
North 19th Way => North 19th Way
West Carol Ann Way => West Carol Ann Way
West Carol Ann Way => West Carol Ann Way
North 7th Way => North 7th Way
North 7th Way => North 7th Way
North 60th Way => North 60th Way
North 6

2:  85007-2616 => 85007
2:  85023-2513 => 85023
2:  85023-2514 => 85023
2:  85225-2412 => 85225
2:  85023-2539 => 85023
2:  85023-2533 => 85023
2:  85023-2532 => 85023
2:  85023-2531 => 85023
2:  85023-2530 => 85023
2:  85023-2537 => 85023
2:  85023-2534 => 85023
2:  8551 => 8551
2:  85023-2384 => 85023
2:  85023-2383 => 85023
2:  85254-4180 => 85254
2:  85021-7217 => 85021
2:  85021-3516 => 85021
2:  85029-3128 => 85029
2:  85013-4403 => 85013
2:  85013-4407 => 85013
2:  85013-4405 => 85013
2:  85118 => 85118
2:  85013-4408 => 85013
2:  85119 => 85119
2:  85226-4880 => 85226
2:  85032-5837 => 85032
2:  85015-3809 => 85015
2:  85023-1402 => 85023
2:  85023-1403 => 85023
2:  85023-1401 => 85023
2:  85219 => 85219
2:  85215 => 85215
2:  85213 => 85213
2:  85212 => 85212
2:  85210 => 85210
2:  85050-4270 => 85050
2:  85023-1519 => 85023
2:  85031-2607 => 85031
2:  85006-3709 => 85006
2:  85023-1514 => 85023
2:  25248 => 25248
2:  85007-2129 => 85007
2:  85006-4460 => 85006
2:  85007-2126 

In [5]:
# Export Data to CSV
import csv
import codecs
import re
import xml.etree.cElementTree as ET
from unittest import TestCase
import cerberus
import schema
import update



# Make sure the fields order in the csvs matches the column order in the
# sql table schema
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

# Shape each element into several data structures
# Clean and shape node or way XML element to Python dict
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    
    node_attribs = {} 
    way_attribs = {}
    way_nodes = []
    tags = []

    if element.tag == 'node':
        for i in NODE_FIELDS:
            node_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):  
            problem = PROBLEMCHARS.search(tag.attrib['k'])
            if not problem:
                node_tag = {} 
                node_tag['id'] = element.attrib['id'] 
                node_tag['value'] = tag.attrib['v']  

                match = LOWER_COLON.search(tag.attrib['k'])
                if not match:
                    node_tag['type'] = 'regular'
                    node_tag['key'] = tag.attrib['k']
                else:
                    bef_colon = re.findall('^(.+):', tag.attrib['k'])
                    aft_colon = re.findall('^[a-z|_]+:(.+)', tag.attrib['k'])
                    node_tag['type'] = bef_colon[0]
                    node_tag['key'] = aft_colon[0]
                    if node_tag['type'] == "addr" and node_tag['key'] == "street":
                        # update street name
                        node_tag['value'] = update.update_name(tag.attrib['v'], mapping) 
                    elif node_tag['type'] == "addr" and node_tag['key'] == "postcode":
                        # update post code
                        node_tag['value'] = update.update_postcode(tag.attrib['v']) 
            tags.append(node_tag)
        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for i in WAY_FIELDS:
            way_attribs[i] = element.attrib[i]
        for tag in element.iter("tag"):
            problem = PROBLEMCHARS.search(tag.attrib['k'])
            if not problem:
                way_tag = {}
                way_tag['id'] = element.attrib['id'] 
                way_tag['value'] = tag.attrib['v']
                match = LOWER_COLON.search(tag.attrib['k'])
                if not match:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = tag.attrib['k']
                else:
                    bef_colon = re.findall('^(.+?):+[a-z]', tag.attrib['k'])
                    aft_colon = re.findall('^[a-z|_]+:(.+)', tag.attrib['k'])

                    way_tag['type'] = bef_colon[0]
                    way_tag['key'] = aft_colon[0]
                    if way_tag['type'] == "addr" and way_tag['key'] == "street":
                        way_tag['value'] = update.update_name(tag.attrib['v'], mapping) 
                    elif way_tag['type'] == "addr" and way_tag['key'] == "postcode":
                        way_tag['value'] = update.update_postcode(tag.attrib['v']) 
            tags.append(way_tag)
        position = 0
        for tag in element.iter("nd"):  
            nd = {}
            nd['id'] = element.attrib['id'] 
            nd['node_id'] = tag.attrib['ref'] 
            nd['position'] = position  
            position += 1
            
            way_nodes.append(nd)
    
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}




# Helper Functions            

# Yield element if it is the right type of tag
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

# Raise ValidationError if element does not match schema
def validate_element(element, validator, schema=SCHEMA):
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

# Extend csv.DictWriter to handle Unicode input
class UnicodeDictWriter(csv.DictWriter, object):
    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# Main Function                    

# Iteratively process each XML element and write to csv(s)
def process_map(file_in, validate):

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()



        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    process_map(OSM_FILE, validate=False)

85345-7050 => 85345
85007-1909 => 85007
85260-5518 => 85260
85023-2502 => 85023
85023-2503 => 85023
85023-2506 => 85023
85023-2507 => 85023
85023-2504 => 85023
85023-2505 => 85023
85284-1103 => 85284
85023-2508 => 85023
85083-4307 => 85083
85083-4302 => 85083
85023-1446 => 85023
85023-1447 => 85023
85023-1444 => 85023
85023-1445 => 85023
85023-1442 => 85023
85023-1443 => 85023
85023-1440 => 85023
85023-1441 => 85023
85023-1520 => 85023
85023-1448 => 85023
85023-1449 => 85023
85021-3542 => 85021
85023-1510 => 85023
85244 => 85244
85247 => 85247
85240 => 85240
85242 => 85242
85032-4748 => 85032
85326 => 85326
85323 => 85323
85249 => 85249
85032-7725 => 85032
85050-7256 => 85050
085028 => 085028
85020 => 85020
85021 => 85021
85022 => 85022
85023 => 85023
85007-2607 => 85007
85023-2301 => 85023
85007-2604 => 85007
85028 => 85028
85029 => 85029
85050-4273 => 85050
85050-4274 => 85050
85023-2309 => 85023
85013-2207 => 85013
85375-4416 => 85375
85351 => 85351
85012-3120 => 85012
85331;85377 =

In [6]:
# Create DB file for SQLite
import csv, sqlite3

con = sqlite3.connect("PHX_AZ.db")
con.text_factory = str
cur = con.cursor()

# create nodes table
cur.execute("CREATE TABLE nodes (id, lat, lon, user, uid, version, changeset, timestamp);")
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp'])              for i in dr]

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp)                 VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
con.commit()

#create nodes_tags table
cur.execute("CREATE TABLE nodes_tags (id, key, value, type);")
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
con.commit()

#Create ways table
cur.execute("CREATE TABLE ways (id, user, uid, version, changeset, timestamp);")
with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
con.commit()

#Create ways_nodes table
cur.execute("CREATE TABLE ways_nodes (id, node_id, position);")
with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?, ?, ?);", to_db)
con.commit()

#Create ways_tags table
cur.execute("CREATE TABLE ways_tags (id, key, value, type);")
with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
con.commit()




In [7]:
# Query DB
import csv, sqlite3

def number_of_nodes():
    result = cur.execute('SELECT COUNT(*) FROM nodes')
    return result.fetchone()[0]

def number_of_ways():
    result = cur.execute('SELECT COUNT(*) FROM ways')
    return result.fetchone()[0]

def number_of_Unique_users():
    result = cur.execute('SELECT COUNT(distinct(uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways)')
    return result.fetchone()[0]

def Top_Contributing_user():
    for row in cur.execute('SELECT e.user, COUNT(*) as num                             FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e                             GROUP BY e.user                             ORDER BY num DESC                             LIMIT 1'):
        return row

def Biggest_religion():
    for row in cur.execute('SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags                             JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="place_of_worship") i                             ON nodes_tags.id=i.id                             WHERE nodes_tags.key="religion"                             GROUP BY nodes_tags.value                             ORDER BY num DESC                            LIMIT 1'):
         return row

def popular_amenity():
    for row in cur.execute('SELECT value, COUNT(*) as num                             FROM nodes_tags                             WHERE key="amenity"                             GROUP BY value                             ORDER BY num DESC                             LIMIT 1'):
        return row

                
if __name__ == '__main__':

    con = sqlite3.connect("PHX_AZ.db") 
    cur = con.cursor()
    print "Number of nodes: " , number_of_nodes()
    print "Number of ways: " , number_of_ways()
    print "Number of unique users: " , number_of_Unique_users()
    print "Top Contributing user: " , Top_Contributing_user()
    print "Biggest religion: " , Biggest_religion()
    print "popular amenity: " , popular_amenity()





Number of nodes:  43512
Number of ways:  5974
Number of unique users:  1231
Top Contributing user:  (u'Dr Kludge', 10705)
Biggest religion:  (u'christian', 3)
popular amenity:  (u'waste_disposal', 27)
